In [1]:
import PyPDF2 as pyPdf
import re 

def hasNumbers(inputString):
    return bool(re.search(r'\s?1\.\s+', inputString))

def hasLatin(inputString):
    return bool(re.search(r'\(x\)', inputString))

def hasBoth (inputString):
    return bool(re.search(r'\s?\d\..*\(i\)', inputString))

def hasSpace (inputString):
    return bool(re.search(r' ', inputString))

In [6]:
pdf_numbers = [3,6,7,11,13,16,22,26,27,30,40,42,48,50,54,56,60,167,264,267,323]
pdf_list = ["C:/Users/KNguyen/pdf_Scraper/document(" + str(x) + ").pdf" for x in pdf_numbers]

In [7]:
print (pdf_list)

['C:/Users/KNguyen/pdf_Scraper/document(3).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(6).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(7).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(11).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(13).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(16).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(22).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(26).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(27).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(30).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(40).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(42).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(48).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(50).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(54).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(56).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(60).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(167).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(264).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(267).pdf', 'C:/Users/KNguyen/p

In [3]:
numbers_list = []
latin_list = []
un_list = []

for pdf in pdf_list:
    doc = pyPdf.PdfFileReader(open(pdf, "rb"))
    count = doc.getNumPages()
    for x in range (0, count):
        one_page = doc.getPage(x).extractText()
        #identify the latin number type pdf
        if hasLatin(one_page) is True:
            latin_list.append(pdf)
            if hasSpace(one_page) is False:
                latin_list.remove(pdf)
                un_list.append(pdf)
                
#put the rest of the pdf in the numbers_list to clean later         
numbers_list = [item for item in pdf_list if item not in latin_list]

In [4]:
for pdf in numbers_list:
    doc = pyPdf.PdfFileReader(open(pdf, "rb"))
    #its only necessary to test on one page; if using first_page, the test fails on doc(40)
    second_page = doc.getPage(1).extractText()
    #identify the unreadable type pdf (pdf without space elements)
    if hasSpace(second_page) is False:
        numbers_list.remove(pdf)
        un_list.append(pdf)

#eliminate duplicates in the unreadable list un_list
un_list = list(set(un_list))

In [5]:
print (un_list)

['C:/Users/KNguyen/pdf_Scraper/document(16).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(1).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(30).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(26).pdf']


In [6]:
print (numbers_list)

['C:/Users/KNguyen/pdf_Scraper/document(3).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(22).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(27).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(40).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(50).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(56).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(60).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(167).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(323).pdf']


In [7]:
print (latin_list)

['C:/Users/KNguyen/pdf_Scraper/document(6).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(7).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(11).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(13).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(42).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(48).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(54).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(264).pdf', 'C:/Users/KNguyen/pdf_Scraper/document(267).pdf']


In [88]:
for pdf in numbers_list:
    doc = pyPdf.PdfFileReader(open(pdf, "rb"))
    count = doc.getNumPages()
    combined_page = ""
    
    for x in range (0, count):
        one_page = doc.getPage(x).extractText()
        match = re.search('[1-9]\..*|\d\d\..*', one_page)
        if match:
            #combined all the pages if match to avoid splitting info
            combined_page += one_page + ""

    #one_clean = re.split('\s+[1-9]\.\s?|\s+\d\d\.\s?', one_page) #get '1.0MW' wrong
    #one_clean = re.split('\s?[1-9]\.\s+|\s+\d\d\.\s?', one_page) #get '2013.' wrong
    one_clean = re.split('\D[1-9]\.\s+|\D1[0-1]\.\s?', combined_page) #this correctly gets '1.25.' and '2013.' but not?? 'aa9.'
    #one_clean = re.split('[\W\D]?[1-9]\.\s+|\D1[0-1]\.\s?', combined_page) # does not get '2013.'
    #one_clean = re.split('\s?\D+[1-9]\.\s+|\D1[0-1]\.\s?', combined_page)
    #print (one_clean)
    for item in one_clean:

        match_add = re.search(r'.*Number of Applicant.*|.*number of the Applicant.*|.*address of facility owner.*', item)
        if match_add:
            desp_1 = match_add.group()
            desp1 = re.split('Number of Applicant|number of the Applicant|address of facility owner', desp_1)
            address = ''
            if (len(desp1)==2):
                address = desp1[1]
            elif (len(desp1)==1):
                address = desp1[0]
            #print ('Desp 1: ' + desp_1)
            #print ("Address: " + address)


        match_owner = re.search(r'.*Individual.*|.*Choose one.*|.*Type of Facility Owner.*', item)
        if match_owner:
            desp_2 = match_owner.group()
            #desp2 = re.split('Individual|is a|Type of Facility Owner', item)
            owner = desp_2
            #print ('Desp_2:' + desp_2)
            #print ('Owner: ' + owner)


        if ('power or fuel' in item):
            desp_3 = item.split('power or fuel')[1]
            match_ac = re.search(r'\d+.*MW \(AC\)|\d+.*\(AC\)|\d+ MWAC|\d+.*MWac|\d+.*MW AC',desp_3)
            if match_ac:
                capacity_ac = match_ac.group()
            else:
                capacity_ac = 'more test!!'
            match_dc = re.search(r'\d+.* \(DC\)|\(\d+.*MWdc\)',desp_3)
            if match_dc:
                capacity_dc = match_dc.group()
            else:
                capacity_dc = 'not available'

            match_type = re.search(r'photovoltaic|Photovoltaic',desp_3)
            if match_type:
                cap_type = match_type.group()
            else:
                cap_type = 'more test!!'
            #print ('Desp 3: ' + desp_3)
            #print ('Capacity_AC: ' + capacity_ac)
            #print ('Capacity_DC: ' + capacity_dc)
            #print ('Type: ' + cap_type)


        match_loc = re.search(r'.*indicated on the map\..*|.*location of generating facility:.*',item)
        if match_loc:
            desp_4 = match_loc.group()
            desp4 = re.split('indicated on the map.|location of generating facility:', desp_4)
            location = desp4[1]
            #print ('Location: ' + location)


        if ('interest in the site' in item):
            #print ('interest: ' + item)
            if ('relation to application' in item):
                desp_5 = item.split('relation to application')[1]
                #print ('Desp 5: ' + desp_5)
            else:
                desp_5 = item.split('interest in the site')[1]
                #print ('Desp 5: ' + desp_5)


        if ('manner of their operation' in item):
            desp_6 = item.split('manner of their operation')[1]
            #print ('Desp 6: ' + desp_6)
            #desp_6 = one_clean[(one_clean.index('(vi)'))+1]
            match_made = re.search(r'(the following)(.*Solar Panels)',desp_6)
            match_made2 = re.search(r'(consisting of)(.*modules)',desp_6)
            match_made3 = re.search(r'(comprised of)(.*modules)',desp_6)
            match_made4 = re.search(r'(consist of)(.*arrays)',desp_6)
            match_made5 = re.search(r'.*modules|.*arrays',desp_6)
            if match_made:
                madeOf = match_made.group(2)
            elif match_made2:
                madeOf = match_made2.group(2)
            elif match_made3:
                madeOf = match_made3.group(2)
            elif match_made4:
                madeOf = match_made4.group(2)
            elif match_made5:
                madeOf = match_made5.group()
            else:
                madeOf = 'not available'

            #match_build = re.search(r'( affixed to )(.*piles)|( affixed to )(.*rackingsystem)',desp_6)
            match_build = re.search(r'(Solar Panels)(.*Racks)',desp_6)
            match_build2 = re.search(r'( attached to )([^.]+)\.',desp_6)
            match_build3 = re.search(r'( facility is )(.*system)',desp_6)
            match_build4 = re.search(r'There will.*purposes',desp_6)
            if match_build:
                build = match_build.group(2)
            elif match_build2:
                build = match_build2.group(2)
            elif match_build3:
                build = match_build3.group(2)
            elif match_build4:
                build = match_build4.group()
            else:
                build = 'more test!!'

            match_in = re.search(r'(Racks)([^.]+inverters)', desp_6)
            match_in2 = re.search(r'(will utilize)([^.]+)\.', desp_6)
            match_in3 = re.search(r'(arrays and)([^.]+inverters)', desp_6)
            match_in4 = re.search(r'.*inverters', desp_6)
            if match_in:
                inverters = match_in.group(2)
            elif match_in2:
                inverters = match_in2.group(2)
            elif match_in3:
                inverters = match_in3.group(2)
            elif match_in4:
                inverters = match_in4.group()
            else:
                inverters = 'not available'

            match_op = re.search(r'(operated by )([^.]+)\.?',desp_6)
            if match_op:
                operator = match_op.group(2)
            else:
                operator = 'not available'
            #print ('Desp 6: ' + desp_6)
            #print ("Made: " + madeOf)
            #print ('Build: ' + build)
            #print ('Inverters: ' + inverters)
            #print ("Operator: " + operator)


        if ('capacity of facility in megawatts' in item):
            desp_7 = item.split('capacity of facility in megawatts')[1]
            #print ('Desp 7: ' + desp_7)


        if ('cost of the facility' in item):
            desp_8 = item.split('cost of the facility')[1]
            match_cost = re.search(r'\$[\s\d,M\.]+',desp_8)
            if match_cost:
                cost = match_cost.group()
            else:
                cost = 'not available'
            #print ('Desp 8: ' + desp_8)
            #print ('Cost: ' + cost)


        if ('the facility will come on line' in item):
            desp_9 = item.split('the facility will come on line')[1]
            match_onl = re.search(r'(is)(.*\d{4})\.?',desp_9)
            #match_onl2 = re.search(r'(on line by)(.*\d{4})\.?',desp_9)
            if match_onl:
                date = match_onl.group(2)
            else:
                date = desp_9
            #print ('Desp_9:' + desp_9)
            #print ('Date: ' + date)


        if ('annual sales in kilowatt hours' in item):
            desp_10 = item.split('annual sales in kilowatt hours')[1]
            match_sell = re.search(r'(electricity.*to)([^.]+)(under)',desp_10)
            match_sell2 = re.search(r'(electricity.*to)([^.]+)\.',desp_10)
            match_sell3 = re.search(r'(contract with)([^.]+)\.',desp_10)
            if match_sell:
                purchaser = match_sell.group(2)
            elif match_sell2:
                purchaser = match_sell2.group(2)
            elif match_sell3:
                purchaser = match_sell3.group(2)
            else:
                purchaser = 'more test!'
            #print ('Desp 10: ' + desp_10)
            #print ('Purchaser: ' + purchaser)


        if ('as soon as they are obtained' in item):
            desp_11 = item.split('as soon as they are obtained.')[1]
            #print ('Desp 11: ' + desp_11)

Desp 11:  The applicant has filed for self-certification as a Qualifying Facility under the Federal Energy Regulatory Commission on August 6, 2014. The applicant has filed a Registration Statement for a New Renewable Energy Facility with the NCUC under Commission R8-66 on July 31,2014. The applicant will file for a Lee County Special Use Permit with the Lee County Planning Board on September, 2014. The applicant will file for an erosion control permit with Lee County Inspections Department and NC DENR in March 2015. The applicant will file for a Lee County Building Permit with the Lee County Inspections Department in June 2015. Cohen Farm Map  SP
-2363, Sub 19
  The pr
oject will be located on PIN: 9645
-07-8678 (outlined below in blue
) on 
Brown Rd. 
in Lee 
County, north
 of Sanford
, NC.  The project will occup
y roughly 45 acres of the 164.22
 acre 
parcel.
   
Desp 11: Permits: Building, Electrical, E&S, Zoning, NCDOT, NCDWQ (as required)FERCform 556 has been submitted (copy atta

In [9]:
for pdf in latin_list:
    doc = pyPdf.PdfFileReader(open(pdf, "rb"))
    count = doc.getNumPages()
    combined_page = ""
    
    for x in range (0, count):
        one_page = doc.getPage(x).extractText()
        match = re.search('\([ivx]+\).*', one_page)
        if match:
            #combined all the pages if match to avoid splitting info
            combined_page += one_page + ""
            
    one_clean = re.split('(\([ivx]+\))', combined_page)
    #print (one_clean)
    for item in one_clean:
        if('(i)' in item):
            desp_1 = one_clean[(one_clean.index('(i)'))+1]
            desp1 = re.split('telephone number are:|facility owner;|owner are listed below:', desp_1)
            address = ''
            if (len(desp1)==2):
                address = desp1[1]
            elif (len(desp1)==1):
                address = desp1[0]
            #print ('Desp 1: ' + desp_1)
            #print ('Address: ' + address)


        if('(ii)' in item):
            desp_2 = one_clean[(one_clean.index('(ii)'))+1]
            #print ('Desp 2: ' + desp_2)


        if('(iii)' in item):
            desp_3 = one_clean[(one_clean.index('(iii)'))+1]
            match_ac = re.search(r'(\d+.*)(\(?AC\)?)|\d+.*MWac|\d+.*MW AC|\d+.*MWAC',desp_3)
            match_ac2 = re.search(r'\d+.*MW',desp_3)
            if match_ac:
                capacity = match_ac.group()
                if ('DC' in capacity):#this test fails if check for 'Grid-tied'
                    match_ac3 = re.search(r'(/)(.+\(AC\))',capacity)
                    if match_ac3:
                        capacity_ac = match_ac3.group(2)
                    else:
                        capacity_ac = 'more test!'
                else:
                    capacity_ac = match_ac.group()
            else:
                capacity_ac = match_ac2.group()

            match_dc = re.search(r'\d+.* \(DC\)',desp_3)
            if match_dc:
                capacity_dc = match_dc.group()
            else:
                capacity_dc = 'not available'

            match_type = re.search(r'photovoltaic|Photovoltaic',desp_3)
            if match_type:
                cap_type = match_type.group()
            else:
                cap_type = 'more test!!'
            #print ('Desp 3: ' + desp_3)
            #print (capacity_ac)
            #print (capacity_dc)
            #print (cap_type)


        if('(iv)' in item):
            desp_4 = one_clean[(one_clean.index('(iv)'))+1]
            match_loc = re.search(r'(located at)(.*\d{5})',desp_4)
            match_loc2 = re.search(r'(located at)(.*NC)\.',desp_4)
            match_loc3 = re.search(r'(located)(.*Carolina)\.',desp_4)
            match_loc4 = re.search(r'(at)(.*\.)',desp_4)
            if match_loc:
                location = match_loc.group(2)
            elif match_loc2:
                location = match_loc2.group(2)
            elif match_loc3:
                location = match_loc3.group(2)
            elif match_loc4:
                location = match_loc4.group(2)
            else:
                location = 'more test!!!'
            #print ('Desp 4: ' + desp_4)
            #print ('Location: ' + location)


        if('(v)' in item):
            desp_5 = one_clean[(one_clean.index('(v)'))+1]
            #print ('Desp 5: ' + desp_5)


        if('(vi)' in item):
            desp_6 = one_clean[(one_clean.index('(vi)'))+1]
            match_made = re.search(r'(consist of)(.*modules)',desp_6)
            match_made2 = re.search(r'.*solar panels',desp_6)
            if match_made:
                madeOf = match_made.group(2)
            elif match_made2:
                madeOf = match_made2.group()
            else:
                madeOf = 'more test!!'

            #match_build = re.search(r'( affixed to )(.*piles)|( affixed to )(.*rackingsystem)',desp_6)
            match_build = re.search(r'( affixed to )([^.]+)\.',desp_6)
            match_build2 = re.search(r'( on )(.*fixed structures)',desp_6)
            if match_build:
                build = match_build.group(2)
            elif match_build2:
                build = match_build2.group(2)
            else:
                build = 'more test!!!'

            match_in = re.search(r'(will utilize)(.+each)', desp_6)
            #put in the same group w/ match_in will return an error
            match_in2 = re.search(r'(will utilize)(.+inverters)', desp_6)
            match_in3 = re.search(r'(connected to)(.+transformers)', desp_6)
            if match_in:
                inverters = match_in.group(2)
            elif match_in2:
                inverters = match_in2.group(2)
            elif match_in3:
                inverters = match_in3.group(2)
            else:
                inverters = 'more test!!!'

            match_op = re.search(r'(operated by )([^.]+)\.?',desp_6)
            if match_op:
                operator = match_op.group(2)
            else:
                operator = 'none'
            print ('Desp 6: ' + desp_6)
            #print ("Made: " + madeOf)
            #print ('Build: ' + build)
            print ('Inverters: ' + inverters)
            #print ("Operator: " + operator)


        if('(vii)' in item):
            desp_7 = one_clean[(one_clean.index('(vii)'))+1]
            #print ('Desp 7: ' + desp_7)


        if('(viii)' in item):
            desp_8 = one_clean[(one_clean.index('(viii)'))+1]
            match_cost = re.search(r'\$[\d,M]+',desp_8)
            if ('projected cost' in desp_8):
                if match_cost:            
                    cost = match_cost.group()
                elif ('confidential' in desp_8):
                    cost = 'confidential'
                else:
                    cost = 'more test!'
            elif ('confidential' in desp_8):
                cost = 'confidential'
            else:
                cost = 'more test!!'
            #print ('Desp 8: ' + desp_8)
            #print ("Cost: " + cost)


        if('(ix)' in item):
            desp_9 = one_clean[(one_clean.index('(ix)'))+1]
            match_onl = re.search(r'(come online in)(.*)\.',desp_9)
            match_onl2 = re.search(r'(scheduled for)(.*)\.',desp_9)
            if match_onl:
                date = match_onl.group(2)
            elif match_onl2:
                date = match_onl2.group(2)
            else:
                date = 'not available'
            #print ('Desp 9: ' + desp_9)
            #print (date)


        if('(x)' in item):
            desp_10 = one_clean[(one_clean.index('(x)'))+1]
            match_sell = re.search(r'(sold to)(.*)under a',desp_10)
            match_sell2 = re.search(r'(sold to)(.*)under',desp_10)
            match_sell3 = re.search(r'(electricity produced to)(.*), ',desp_10)
            if match_sell:
                purchaser = match_sell.group(2)
            elif match_sell2:
                purchaser = match_sell2.group(2)
            elif match_sell3:
                purchaser = match_sell3.group(2)
            else:
                purchaser = 'more test!'
            #print ('Desp 10: ' + desp_10)
            #print ('Purchaser: ' + purchaser)


        if('(xi)' in item):
            desp_11 = one_clean[(one_clean.index('(xi)'))+1]
            #print ('Desp 11: ' + desp_11)

Desp 6:  The Facility will consist of 72-cell polycrystalline PV modules (specificationand quantity to be determined) affixed to ground mounted racks supportedon driven piles. The system will utilize utility grade inverters, at least 500kWeach, and will be interconnected to the grid operated by Duke Energy Progress("PEP"). The Facility will connect with DEP via 115kV transmission or 23kV distribution (both are on site and has not yet been determined). Arepresentative site layout is included as Attachment 2, filed confidentially.
Inverters:  utility grade inverters, at least 500kWeach
Desp 6:  The Facility will consist of 72-cell polycrystalline PV modules (specificationand quantity to be determined) affixed to ground mounted racks supportedon driven piles. The system will utilize utility grade inverters, at least 500kWeach, and will be interconnected to the grid operated by Duke Energy Progress("PEP"). The Facility will connect with DEP via 115kV transmission or 23kV distribution (both